In [1]:
from data_gen_utils import *
import sys
sys.path.append("../")

import numpy as np
import torch
import matplotlib.pyplot as plt
import mpmath

from metal.label_model import LabelModel, CliqueTree
from metal.label_model.utils import (
    compute_mu,
    compute_covariance,
    compute_inv_covariance,
    print_matrix,
    visualize_matrix
)

/lfs/1/annhe/anaconda3/envs/metal/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
M = 6
n = 10000
K = 1
dg = DataGenerator(M,n)

In [3]:
dg.generate_O(k=3)
print(dg.O.shape)
print(dg.L.shape)

(6, 6)
(6, 10000)


In [4]:
print(dg.deps)

[(0, 1), (2, 3), (4, 5)]


In [5]:
w,h = (dg.L).shape
L = np.zeros((w,h),dtype=float)
for i in range(w):
    for j in range(h):
        if dg.L[i,j] > 0:
            L[i,j] = 1
        else:
            L[i,j] = 0

In [6]:
lm = LabelModel(k=1)
lm.config['higher_order_cliques'] = False
lm.config['verbose'] = True

1


In [7]:
lm.train(L=L.T,
         deps=dg.deps_no_diag,
         all_unary_cliques=True,
        higher_order_cliques=False,
        n_epochs=50000,
        print_every=5000,
        lr=1e-6,
        l2=0.1,
        O_inv_prec=1024,)

Computing O^{-1}...
> /lfs/1/annhe/metal/metal/label_model/label_model.py(284)_generate_O_inv()
-> O_inv = (O_unnorm / n) ** -1
(Pdb) c
O INV SHAPE:  (6, 6)
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {1}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {2}
ci max cliques  {0}
ci max cliques  {0}
ci max cliques  {0}
ci max cliques  {0}
ci max cliques  {0}
ci max cliques  {0}
ci 

In [8]:
R_mu_est = lm.mu.detach().numpy()
print(R_mu_est)
print(dg.mu)

[[0.63837886]
 [0.31835738]
 [0.5274937 ]
 [0.38369054]
 [0.4385993 ]
 [0.262882  ]]
[0.4848 0.6346 0.6958 0.677  0.2876 0.5678]
